In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
import time

In [2]:
lista_ligas = {"2015-2016": "https://fbref.com/es/comps/12/2015-2016/Estadisticas-2015-2016-La-Liga"}

In [3]:
lista_equipos = {}
for a, x in lista_ligas.items():
    options = Options()
    options.add_argument("--start-maximized")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    url = x
    driver.get(url)
    driver.implicitly_wait(10)

    # Buscar todos los enlaces de equipos
    tabla = driver.find_element(By.CSS_SELECTOR, "table.stats_table.sortable.min_width.force_mobilize.now_sortable")
    enlaces = tabla.find_elements(By.CSS_SELECTOR, 'a[href*="/es/equipos/"]')

    equipos = []

    for enlace in enlaces:
        href = enlace.get_attribute("href")
        texto = enlace.text.strip()
        
        partes = href.split("/")

        try:
            idx = partes.index("equipos")
            id_equipo = partes[idx + 1]  # El ID está justo después de "equipos"
            equipos.append(id_equipo)
        except (ValueError, IndexError):
            continue  # Si no encuentra "equipos" o el ID está mal formado
    driver.quit()
    time.sleep(5)
    lista_equipos[a] = equipos


In [4]:
response = requests.post('https://fbrapi.com/generate_api_key')
api_key = response.json()['api_key']

In [8]:
url = "https://fbrapi.com/teams"
jugadores = {}
no_añadidos = {}
for x, a in lista_equipos.items():
    params = {
        "team_id": a,
        "season_id": x
        }
    headers = {"X-API-Key": api_key}
    response = requests.get(url, params=params, headers=headers)
    print(f"equipo {a}, año {x}, {response.status_code}")
    if response.status_code == 200:
        jugadores[x] = response.json()
    else:
        if x not in no_añadidos:
            no_añadidos[x] = []
        no_añadidos[x].append(a)
    time.sleep(6)
print(jugadores)

equipo ['206d90db', '53a2f082', 'db3b9613', '2a8183b3', '2b390eca', 'f25da7fb', 'ad2be733', '1c896955', 'e31d1cd9', 'fc536746', '0049d422', 'dcc91a7b', 'a8661628', 'bea5c710', '2a60ed82', 'a0435291', 'bb9efd50', '98e8af82', '7848bd64', '9800b6a1'], año 2015-2016, 200
{'2015-2016': {'team_roster': {'data': [{'player': 'Luis Suárez', 'player_id': 'a6154613', 'nationality': 'URU', 'position': 'FW', 'age': 28, 'mp': 35, 'starts': 35}, {'player': 'Neymar', 'player_id': '69384e5d', 'nationality': 'BRA', 'position': 'FW,MF', 'age': 23, 'mp': 34, 'starts': 34}, {'player': 'Sergio Busquets', 'player_id': '5ab0ea87', 'nationality': 'ESP', 'position': 'MF', 'age': 27, 'mp': 35, 'starts': 34}, {'player': 'Claudio Bravo', 'player_id': '10072610', 'nationality': 'CHI', 'position': 'GK', 'age': 32, 'mp': 32, 'starts': 32}, {'player': 'Lionel Messi', 'player_id': 'd70ce98e', 'nationality': 'ARG', 'position': 'FW,MF', 'age': 28, 'mp': 33, 'starts': 31}, {'player': 'Javier Mascherano', 'player_id': '507

In [9]:
url = "https://fbrapi.com/teams"
jugadores = {}
no_añadidos = {}
headers = {"X-API-Key": api_key}

for temporada, lista_ids in lista_equipos.items():  # lista_ids es una lista de team_ids
    for team_id in lista_ids:
        params = {
            "team_id": team_id,
            "season_id": temporada
        }
        try:
            response = requests.get(url, params=params, headers=headers)
            print(f"Equipo {team_id}, Año {temporada}, Status: {response.status_code}")

            if response.status_code == 200:
                if temporada not in jugadores:
                    jugadores[temporada] = {}  # lista de equipos para esa temporada
                jugadores[temporada][team_id] = response.json()
            else:
                if temporada not in no_añadidos:
                    no_añadidos[temporada] = {}
                no_añadidos[temporada].append(team_id)

        except Exception as e:
            print(f"Error en temporada {temporada}, equipo {team_id}: {e}")
            if temporada not in no_añadidos:
                no_añadidos[temporada] = {}
            no_añadidos[temporada].append(team_id)
        time.sleep(6)  # por si hay límite de llamadas


Equipo 206d90db, Año 2015-2016, Status: 200
Equipo 53a2f082, Año 2015-2016, Status: 200
Equipo db3b9613, Año 2015-2016, Status: 200
Equipo 2a8183b3, Año 2015-2016, Status: 200
Equipo 2b390eca, Año 2015-2016, Status: 200
Equipo f25da7fb, Año 2015-2016, Status: 200
Equipo ad2be733, Año 2015-2016, Status: 200
Equipo 1c896955, Año 2015-2016, Status: 200
Equipo e31d1cd9, Año 2015-2016, Status: 200
Equipo fc536746, Año 2015-2016, Status: 200
Equipo 0049d422, Año 2015-2016, Status: 200
Equipo dcc91a7b, Año 2015-2016, Status: 200
Equipo a8661628, Año 2015-2016, Status: 200
Equipo bea5c710, Año 2015-2016, Status: 200
Equipo 2a60ed82, Año 2015-2016, Status: 200
Equipo a0435291, Año 2015-2016, Status: 200
Equipo bb9efd50, Año 2015-2016, Status: 200
Equipo 98e8af82, Año 2015-2016, Status: 200
Equipo 7848bd64, Año 2015-2016, Status: 200
Equipo 9800b6a1, Año 2015-2016, Status: 200


In [10]:
df_jugadores = []
for x, i in jugadores.items():  # x = temporada, i = diccionario de equipos
    for a, b in i.items():      # a = team_id, b = contenido (team_roster, team_schedule)
        c = b["team_roster"]
        for d, e in c.items():  # c debería ser "data"
            for f in e:  # e = jugador (diccionario)
                f["season_id"] = x
                f["team_id"] = a
                df_jugadores.append(f)

# Crear el DataFrame
df_jugadores = pd.DataFrame(df_jugadores)  

In [20]:
url = "https://fbrapi.com/player-season-stats"
headers = {"X-API-Key": api_key}
data_jugadores = {}
for x, i in lista_equipos.items():
    for team_id in i:
        params = {
        "team_id": team_id,
        "league_id": 12,
        "season_id": "2015-2016"}
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data_jugadores[team_id] = response.json()
        time.sleep(3)


In [ ]:
for team_id, team_info in data_jugadores.items():
    print(team_info)
    for player in team_info['players']:
        print(player)

In [ ]:
data_jugadores

In [32]:
# Lista para almacenar los datos de cada jugador
players_data = []

# Iteramos sobre cada equipo en el diccionario
for team_id, team_info in data_jugadores.items():
    # Iteramos sobre cada jugador en el equipo
    for player in team_info['players']:
        # Creamos un diccionario base con el ID del equipo
        player_dict = {'team_id': team_id}
        
        # Añadimos todos los campos de meta_data con prefijo 'meta_'
        for key, value in player['meta_data'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos todos los campos de stats.stats con prefijo 'stats_'
        for key, value in player['stats']['stats'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos todos los campos de stats.playingtime con prefijo 'playingtime_'
        for key, value in player['stats']['playingtime'].items():
            player_dict[f'{key}'] = value
        
        # Añadimos este jugador a la lista
        players_data.append(player_dict)

# Creamos el DataFrame
df_info = pd.DataFrame(players_data)

In [34]:
df_info.columns

Index(['team_id', 'player_id', 'player_name', 'player_country_code', 'age',
       'positions', 'matches_played', 'starts', 'min', 'gls', 'ast',
       'non_pen_gls', 'pk_made', 'pk_att', 'yellow_cards', 'red_cards',
       'per90_gls', 'per90_ast', 'per90_non_pen_gls', 'min_per_match_played',
       'pct_squad_min', 'avg_min_starter', 'subs', 'avg_min_sub', 'unused_sub',
       'team_gls_on_pitch', 'team_gls_ag_on_pitch', 'per90_plus_minus',
       'per90_on_off'],
      dtype='object')

In [36]:
df_info = df_info.dropna(subset=['player_id'])

In [38]:
df_info.to_csv("data/jugadores_LaLiga_completo.csv", index = False)